In [ ]:
import os
import gc
import json
import cv2
import time
import shutil
from PIL import Image
import pandas as pd
import numpy as np
import pydicom
import glob
from pydicom.pixel_data_handlers.util import apply_voi_lut
import tensorflow as tf
import tensorflow_addons as tfa
import matplotlib.pyplot as plt
import albumentations as A
from sklearn.model_selection import StratifiedKFold
import tensorflow.keras.backend as K
from tensorflow.keras import Model, Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import Sequence
from tensorflow.keras.losses import BinaryCrossentropy, CategoricalCrossentropy
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import *
from tensorflow.keras.metrics import AUC, CategoricalAccuracy
from tqdm import tqdm
import efficientnet.tfkeras as efn
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
print('tensorflow version:', tf.__version__)

In [ ]:
KAGGLE = False
MDLS_FOLDS = {'v10': [0, 1]}
MDLS_FOLDS_TWOCLS = {'vbin1': [0, 1]}
if KAGGLE:
    DATA_PATH = '../input/siim-covid19-detection'
    MDLS_PATHS = {ver: f'../input/siim-tfmodels-{ver}' 
                  for ver, _ in MDLS_FOLDS.items()}
    MDLS_PATHS_TWOCLS = {ver: f'../input/siim-tfmodels-{ver}' 
                         for ver, _ in MDLS_FOLDS_TWOCLS.items()}
else:
    DATA_PATH = './data'
    MDLS_PATHS = {ver: f'./models_{ver}' 
                  for ver, _ in MDLS_FOLDS.items()}
    MDLS_PATHS_TWOCLS = {ver: f'./models_{ver}' 
                         for ver, _ in MDLS_FOLDS_TWOCLS.items()}
CACHE_PATHS = {ver: './cache' for ver, _ in MDLS_FOLDS.items()}
CACHE_PATHS_TWOCLS = {ver: './cache' for ver, _ in MDLS_FOLDS_TWOCLS.items()}
TTAS = [0, 1]

start_time = time.time()

# All classes model infer

In [ ]:
params_dict = {}
for ver, _ in MDLS_FOLDS.items():
    with open(f'{MDLS_PATHS[ver]}/params.json') as file:
        params_dict[ver] = json.load(file)
for ver, params in params_dict.items():
    print('version:', ver, '| loaded params:', params, '\n')

In [ ]:
def read_xray(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to 
    # "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array        
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8) 
    return data

def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    img = Image.fromarray(array)
    if keep_ratio:
        img.thumbnail((size, size), resample)
    else:
        img = img.resize((size, size), resample)
    return img

In [ ]:
filepaths = glob.glob(f'{DATA_PATH}/test/**/*dcm', recursive=True)
test_df = pd.DataFrame({'path': filepaths,})
test_df['image_id'] = test_df.path.map(
    lambda x: x.split('/')[-1].replace('.dcm', '') 
    + '_image'
)
test_df['study_id'] = test_df.path.map(
    lambda x: x.split('/')[-3].replace('.dcm', '') 
    + '_study'
)
display(test_df.head())
print('test df loaded', test_df.shape)

In [ ]:
for ver, params in params_dict.items():
    counter = 0
    images_paths = []
    dim_x = []
    dim_y = []
    os.makedirs(CACHE_PATHS[ver], exist_ok=True)
    for file in tqdm(test_df.path, desc=f'test {ver}'):
        if file == '':
            counter += 1
        else:
            xray = read_xray(file)
            im = resize(xray, size=params['img_size']) # keep_ratio=True to have original aspect ratio
            im.save(CACHE_PATHS[ver] + '/' + file.split('/')[-1].replace('dcm', 'png'))
            images_paths.append(file.split('/')[-1].replace('dcm', 'png'))
            dim_x.append(xray.shape[1])
            dim_y.append(xray.shape[0])
    print('files omitted:', counter)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
test_df['img'] = images_paths
test_df['dim_x'] = dim_x
test_df['dim_y'] = dim_y
display(test_df.head())
print('test df done', test_df.shape)

In [ ]:
EFNS = [efn.EfficientNetB0, efn.EfficientNetB1, 
        efn.EfficientNetB2, efn.EfficientNetB3, 
        efn.EfficientNetB4, efn.EfficientNetB5, 
        efn.EfficientNetB6, efn.EfficientNetB7]

def get_model(params, classes=4, lr=.001, lbl_smth=.0001):
    input_shape=(params['img_size'], params['img_size'], 3)
    enet = EFNS[params['backbone']](
        input_shape=input_shape,
        weights='imagenet',
        include_top=False
    )
    inp = Input(shape=input_shape)
    x = enet(inp)
    x = GlobalAveragePooling2D()(x)
    x = Dense(64, activation = 'relu')(x)
    if classes == 1:
        x = Dense(classes, activation='sigmoid')(x)
        loss = BinaryCrossentropy(label_smoothing=params['lbl_smth'])
        auc = tf.keras.metrics.AUC(name='auc')
        accuracy = 'accuracy'
        f1  = tfa.metrics.F1Score(
            num_classes=classes, 
            average='macro', 
            threshold=None
        )
    else:
        x = Dense(classes, activation='softmax')(x)
        loss = CategoricalCrossentropy(label_smoothing=params['lbl_smth'])
        auc = AUC(name='auc', curve='ROC', multi_label=True)
        accuracy = CategoricalAccuracy()
        f1  = tfa.metrics.F1Score(
            num_classes=classes, 
            average='macro', 
            threshold=None
        )
    model = Model(inputs=inp, outputs=x)
    model.compile(
        optimizer=tfa.optimizers.Lookahead(
            tf.keras.optimizers.Adam(learning_rate=params['lr']),
            sync_period=max(6, int(params['patience'] / 4))
        ),
        loss=loss, 
        metrics=[auc, accuracy, f1]
    )
    return model

In [ ]:
class DataGenSIIM(Sequence):
    
    def __init__(self, df, classes, imgs_path, imgs_idxs, img_size,
                 batch_size=8, mode='fit', shuffle=False, aug=None, 
                 resize=None, tta=0, two_cls=False):
        self.df = df
        self.classes = classes
        self.imgs_path = imgs_path
        self.imgs_idxs = imgs_idxs
        self.img_size = img_size
        self.batch_size = batch_size
        self.mode = mode
        self.shuffle = shuffle
        self.aug = aug
        self.resize = resize
        self.tta = tta
        self.two_cls = two_cls
        self.on_epoch_end()
        
    def __len__(self):
        return int(np.ceil(len(self.imgs_idxs) / self.batch_size))
    
    def on_epoch_end(self):
        self.indexes = np.arange(len(self.imgs_idxs))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __getitem__(self, index):
        batch_size = min(self.batch_size, len(self.imgs_idxs) - index*self.batch_size)
        X = np.zeros((batch_size, self.img_size, self.img_size, 3), dtype=np.float32)
        imgs_batch = self.imgs_idxs[index * self.batch_size : (index+1) * self.batch_size]
        if self.mode == 'fit':
            if self.two_cls:
                y = np.zeros(batch_size, dtype=np.float32)
            else:
                y = np.zeros((batch_size, len(self.classes)), dtype=np.float32)
            for i, img_idx in enumerate(imgs_batch):
                X[i, ], y[i] = self.get_img(img_idx)
            return X, y
        elif self.mode == 'predict':
            for i, img_idx in enumerate(imgs_batch):
                X[i, ] = self.get_img(img_idx)
            return X
        else:
            raise AttributeError('fit mode parameter error')
            
    def get_img(self, img_idx):
        img_path = f'{self.imgs_path}/{img_idx}'
        img = cv2.imread(img_path)
        if img is None:
            print('error load image:', img_path)
        if self.resize:
            img = cv2.resize(img, (int(img.shape[1] / self.resize), int(img.shape[0] / self.resize)))
        img = img.astype(np.float32) / 255
        if self.mode == 'fit':
            if self.two_cls:
                label = self.df.loc[self.df['img'] == img_idx, 'None Opacity'].values[0]
            else:
                label = self.df.loc[self.df['img'] == img_idx, self.classes].values[0]
            if label is None:
                print('error load label:', img_path)
            label = label.astype(np.float32)
            if self.aug:
                img = self.aug(image=img)['image']
            return img, label
        else:
            if self.aug:
                img = self.aug(image=img)['image']
            img = self.flip(img, axis=self.tta)
            return img
        
    def flip(self, img, axis=0):
        if axis == 1:
            return img[::-1, :, ]
        elif axis == 2:
            return img[:, ::-1, ]
        elif axis == 3:
            return img[::-1, ::-1, ]
        else:
            return img

In [ ]:
BATCH_SUBM = 64
imgs_idxs = test_df.img.values
preds = []
for ver, folds in MDLS_FOLDS.items():
    models = []
    for n_fold in folds:
        checkpoint_path = f'{MDLS_PATHS[ver]}/model_{n_fold}.hdf5'
        model = get_model(
            params_dict[ver]
        )
        model.load_weights(checkpoint_path)
        models.append(model)
        print('ver', ver, '-> model loaded', checkpoint_path)
    for tta in TTAS:
        print(f'ver {ver} classes {params_dict[ver]["classes"]}')
        test_datagen = DataGenSIIM(
            df=test_df,
            classes=params_dict[ver]['classes'],
            imgs_path=CACHE_PATHS[ver], 
            imgs_idxs=imgs_idxs, 
            img_size=params_dict[ver]['img_size'], 
            batch_size=BATCH_SUBM, 
            mode='predict', 
            shuffle=False,           
            aug=None, 
            resize=None,
            tta=tta
        )
        for i, model in enumerate(models):
            preds.append(model.predict(test_datagen))
            print(f'ver {ver} | tta {tta} | model {i} -> prediction done')
    del models; gc.collect()
preds = np.array(np.mean(preds, axis=0))
print('all done | preds shape:', preds.shape)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
name2fname = {
    'Negative for Pneumonia': 'negative', 
    'Typical Appearance': 'typical', 
    'Indeterminate Appearance': 'indeterminate', 
    'Atypical Appearance': 'atypical'
}
name2label = {v: i for i, (k, v) in enumerate(name2fname.items())}
print(name2label)
label2name  = {v:k for k, v in name2label.items()}
print(label2name)

In [ ]:
cols_classes = [str(x) for x in list(name2label.values())]
for i, col in enumerate(cols_classes):
    test_df[col] = preds[:, i]
display(test_df.head())
print('test df study part done', test_df.shape)

# 2 class model infer

In [ ]:
params_dict = {}
for ver, _ in MDLS_FOLDS_TWOCLS.items():
    with open(f'{MDLS_PATHS_TWOCLS[ver]}/params.json') as file:
        params_dict[ver] = json.load(file)
for ver, params in params_dict.items():
    print('version:', ver, '| loaded params:', params, '\n')

In [ ]:
BATCH_SUBM = 64
imgs_idxs = test_df.img.values
preds = []
for ver, folds in MDLS_FOLDS_TWOCLS.items():
    models = []
    for n_fold in folds:
        checkpoint_path = f'{MDLS_PATHS_TWOCLS[ver]}/model_{n_fold}.hdf5'
        model = get_model(
            params_dict[ver],
            classes=1
        )
        model.load_weights(checkpoint_path)
        models.append(model)
        print('ver', ver, '-> model loaded', checkpoint_path)
    for tta in TTAS:
        print(f'ver {ver} classes {params_dict[ver]["classes"]}')
        test_datagen = DataGenSIIM(
            df=test_df,
            classes=params_dict[ver]['classes'],
            imgs_path=CACHE_PATHS_TWOCLS[ver], 
            imgs_idxs=imgs_idxs, 
            img_size=params_dict[ver]['img_size'], 
            batch_size=BATCH_SUBM, 
            mode='predict', 
            shuffle=False,           
            aug=None, 
            resize=None,
            tta=tta,
            two_cls=True
        )
        for i, model in enumerate(models):
            preds.append(model.predict(test_datagen))
            print(f'ver {ver} | tta {tta} | model {i} -> prediction done')
    del models; gc.collect()
preds = np.array(np.mean(preds, axis=0))
print('all done | preds shape:', preds.shape)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

In [ ]:
test_df['None Opacity'] = preds
display(test_df.head())
print('test df study part done', test_df.shape)

# MMDet model infer

In [ ]:
import mmcv
import mmdet
from numba import cuda
from ensemble_boxes import *
from mmdet.apis import init_detector, inference_detector
print(mmdet.__version__)

In [ ]:
cuda.select_device(0)
cuda.close()

In [ ]:
BXS = True
VER_BXS = 'v0'
if KAGGLE:
    MDLS_BXS_PATH = f'../input/siim-mmdetection-train-demo' 
else:
    MDLS_BXS_PATH = f'/u01/mrorange/siim/models_mmdet_{VER_BXS}'
TH = .35
IOU_TH = .6

In [ ]:
with open(f'{MDLS_BXS_PATH}/params.json') as file:
    params_bxs = json.load(file)
print(params_bxs)

In [ ]:
checkpoint = f'{MDLS_BXS_PATH}/epoch_6.pth'
cfg = f'{MDLS_BXS_PATH}/init_config.py'
model_bxs = init_detector(cfg, checkpoint, device='cuda:0')

In [ ]:
def plot_imgs(imgs, cols=2, size=10, is_rgb=True, title='', cmap='gray', img_size=None):
    rows = len(imgs) // cols + 1
    fig = plt.figure(figsize=(cols * size, rows * size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i + 1)
        plt.axis('off')
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.axis('off')
    
def draw_bbox(img, box, label, color, thickness=3):   
    alpha = .1
    alpha_box = .4
    overlay_bbox = img.copy()
    overlay_text = img.copy()
    output = img.copy()
    text_width, text_height = cv2.getTextSize(label.upper(), cv2.FONT_HERSHEY_SIMPLEX, .6, 1)[0]
    cv2.rectangle(overlay_bbox, 
                  (box[0], box[1]), 
                  (box[2], box[3]), 
                  color, -1)
    cv2.addWeighted(overlay_bbox, alpha, output, 1 - alpha, 0, output)
    cv2.rectangle(overlay_text, 
                  (box[0], box[1] - 7 - text_height), 
                  (box[0] + text_width + 2, box[1]),
                  (0, 0, 0), -1)
    cv2.addWeighted(overlay_text, alpha_box, output, 1 - alpha_box, 0, output)
    cv2.rectangle(output, 
                  (box[0], box[1]), 
                  (box[2], box[3]),
                  color, thickness)
    cv2.putText(output, 
                label.upper(), 
                (box[0], box[1]-5),
                cv2.FONT_HERSHEY_SIMPLEX, 
                .6, (255, 255, 255), 1, 
                cv2.LINE_AA)
    return output

In [ ]:
imgs = []
preds_bxs = []
for ver, params in params_dict.items():
    for i, img_name in enumerate(tqdm(imgs_idxs, desc=f'test {ver}')):
        ratio_x = params['img_size'] / test_df.loc[test_df['img'] == img_name, 'dim_x'].values[0]
        ratio_y = params['img_size'] / test_df.loc[test_df['img'] == img_name, 'dim_y'].values[0]
        img = cv2.imread(f'{CACHE_PATHS_TWOCLS[ver]}/{img_name}')
        result = inference_detector(model_bxs, img)
        boxes_list = [list(x[:, :4] / params['img_size']) for x in result if x.shape[0] != 0]
        boxes_list =  [item for sublist in boxes_list for item in sublist]
        scores_list = [x[:, 4].tolist() for x in result if x.shape[0] != 0]
        scores_list =  [item for sublist in scores_list for item in sublist]
        labels_list = [[i] * x.shape[0] for i, x in enumerate(result) if x.shape[0] != 0]
        labels_list =  [item for sublist in labels_list for item in sublist]
        boxes, scores, box_labels = nms(
            boxes=[boxes_list], 
            scores=[scores_list], 
            labels=[labels_list], 
            weights=None,
            iou_thr=IOU_TH
        )
        boxes *= params['img_size']
        if i <= 3:
            for label_id, box, score in zip(box_labels, boxes, scores):
                if score >= TH:
                    color = [255, 255, 255]
                    img = draw_bbox(
                        img, 
                        list(np.int_(box)), 
                        'predict', 
                        color
                    )
            imgs.append(img)
        string = ''
        for label_id, box, score in zip(box_labels, boxes, scores):
            if score >= TH:
                str_boxes = ' '.join([
                    str(int(box[0] / ratio_x)),
                    str(int(box[1] / ratio_y)),
                    str(int(box[2] / ratio_x)),
                    str(int(box[3] / ratio_y))
                ])
                string += f'opacity {score:0.2f} {str_boxes} '
        string = string.strip()
        preds_bxs.append(string if string else 'none 1 0 0 1 1')
    plot_imgs(imgs, size=8, cols=4, cmap=None)

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')

# Post-processing for submission

In [ ]:
if BXS:
    image_df = pd.DataFrame({
        'image_id': test_df.image_id.tolist(),
        'PredictionString_img': preds_bxs
    })
else:
    image_df = pd.DataFrame({
        'image_id': test_df.image_id.tolist(),
        'PredictionString_img': ["none 1 0 0 1 1"] * len(test_df.image_id.tolist())
    })
display(image_df.head())
print('image df done', image_df.shape)

In [ ]:
def get_predstring(row, thr=0):
    string = ''
    for idx in range(4):
        conf =  row[str(idx)]
        if conf > thr:
            string += f'{label2name[idx]} {conf:0.2f} 0 0 1 1 '
    string = string.strip()
    return string

def get_prednoneop(row):
    string = row['PredictionString_img']
    if string != 'none 1 0 0 1 1':
        string += f' none {row["None Opacity"]:0.2f} 0 0 1 1'
    string = string.strip()
    return string

def get_negative(row):
    string = row['PredictionString_img']
    if np.argmax(row[cols_classes]) == 0:
        string = 'none 0 0 1 1'
    string = string.strip()
    return string

cols = ['image_id', 'study_id']
print('test shape:', test_df.shape)
subm_df = pd.merge(test_df[cols], image_df, 
                   left_on='image_id', right_on='image_id', 
                   how='left')
subm_df = pd.merge(subm_df, 
                   test_df.groupby(['study_id'])[cols_classes].mean().reset_index(), 
                   left_on='study_id', right_on='study_id', 
                   how='left')
subm_df = pd.merge(subm_df, 
                   test_df.groupby(['study_id'])['None Opacity'].mean().reset_index(), 
                   left_on='study_id', right_on='study_id', 
                   how='left')
subm_df['PredictionString_sty'] = subm_df.apply(get_predstring, axis=1)
subm_df['PredictionString_img'] = subm_df.apply(get_prednoneop, axis=1)
subm_df['PredictionString_img'] = subm_df.apply(get_negative, axis=1)
display(subm_df.head())
print('unique study ids:', len(subm_df.study_id.unique()))
print('unique image ids:', len(subm_df.image_id.unique()))
print('subm shape:', subm_df.shape)

In [ ]:
subm_df = pd.concat([
    subm_df[['image_id', 'PredictionString_img']].rename(
        columns={'image_id': 'id', 'PredictionString_img': 'PredictionString'}
    ).drop_duplicates(), 
    subm_df[['study_id', 'PredictionString_sty']].rename(
        columns={'study_id': 'id', 'PredictionString_sty': 'PredictionString'}
    ).drop_duplicates()
])
subm_df.to_csv('submission.csv', index=False)
display(subm_df.head())
print('submission done:', subm_df.shape)

In [ ]:
for ver, cache_path in CACHE_PATHS.items():
    shutil.rmtree(cache_path)
    print(f'ver {ver} | path {cache_path} -> cache deleted')

elapsed_time = time.time() - start_time
print(f'time elapsed: {elapsed_time // 60:.0f} min {elapsed_time % 60:.0f} sec')